<a href="https://colab.research.google.com/github/BYU-Hydroinformatics/gwdm-notebooks/blob/main/GWDM_AquiferMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages

In [ ]:
#@markdown ### **Installing packages**
%%capture
!pip install geopandas
!pip install rasterio
!pip install rioxarray
!pip install gstools
!pip install matplotlib
#!pip install kats
!pip install pyod
!pip install netCDF4

In [ ]:
#@markdown ### **Importing packages**
import calendar
import copy
import datetime
import math
import os
import shutil
import tempfile
import time
import urllib
from pathlib import Path
from timeit import default_timer as timer
from urllib import request
from xml.etree import cElementTree as ET

import geopandas as gpd
import gstools as gs
import netCDF4
import numpy as np
import pandas as pd
import requests
import xarray
import rioxarray
from scipy import interpolate
from shapely import wkt
from shapely.geometry import mapping
from google.colab import files
import matplotlib.pyplot as plt
import re
import ipywidgets as widgets

# Uploading Aquifer Data

In [ ]:
#@markdown ### **Uploading Aquifer File**
upload_aquifers = files.upload()
aquifers = gpd.GeoDataFrame.from_file(list(upload_aquifers.keys())[0])

aList1 = list(aquifers.columns)
#aList1.append('NA')
a_ID = widgets.Dropdown(options=aList1, description = "Aquifer ID")
a_name = widgets.Dropdown(options=aList1, description = "Aquifer Name")

a1Items = [a_ID, a_name]
print("\nPlease select the appropriate headers for your file")
widgets.GridBox(a1Items, layout=widgets.Layout(grid_template_columns="repeat(1, 550px)"))

In [ ]:
#@markdown ### **Select the Aquifer you would like to Map**
aList2 = list(aquifers[a_name.value])
aquifer_name = widgets.Dropdown(options=aList2, description = "Aquifer Name")

a2Items = [aquifer_name]
print("\nPlease select the aquifer you would like to map")
widgets.GridBox(a2Items, layout=widgets.Layout(grid_template_columns="repeat(1, 550px)"))

In [ ]:
#@markdown ### **Filter and Display Aquifer Information**
aquifer_index = aquifers[aquifers[a_name.value]==aquifer_name.value].index.values
aquifer = aquifers[aquifers[a_name.value].isin([aquifer_name.value])]
aquifer_ID = list(aquifer[a_ID.value])
aquifer

In [ ]:
#@markdown ### **Uploading wells file**
upload_wells = files.upload()
wells = pd.read_csv(list(upload_wells.keys())[0])

wList = list(wells.columns)
wList.append('NA')
w_well_ID = widgets.Dropdown(options=wList, description = "Well ID")
w_well_name = widgets.Dropdown(options=wList, description = "Well Name")
w_lat = widgets.Dropdown(options=wList, description = "Latitude")
w_long = widgets.Dropdown(options=wList, description = "Longitude")
w_aquifer_ID = widgets.Dropdown(options=wList, description = "Aquifer ID")

wItems = [w_well_ID, w_well_name, w_lat, w_long, w_aquifer_ID]
print("\nPlease select the appropriate headers for your file")
widgets.GridBox(wItems, layout=widgets.Layout(grid_template_columns="repeat(1, 550px)"))

In [ ]:
#To avoid type mismatch, convert the aquifer_id found from the aquifer file and the aquiferid column
#in the wells dataframe to both be strings
aquifer_ID = [str(item) for item in aquifer_ID]
if (w_aquifer_ID.value !="NA"):
  wells[w_aquifer_ID.value] = wells[w_aquifer_ID.value].astype(str)

In [ ]:
#@markdown ### **Formatting wells dataframe**
wells_gdf = gpd.GeoDataFrame(wells, geometry=gpd.points_from_xy(wells[w_long.value], wells[w_lat.value]))
if (w_aquifer_ID.value !="NA"):
  wells_gdf = wells_gdf[wells_gdf[w_aquifer_ID.value].isin(aquifer_ID)]
wells_gdf

In [ ]:
#@markdown ### **Plotting aquifer and wells**
fig, ax = plt.subplots()
aquifer.plot(color="none", edgecolor="red", ax=ax)
wells_gdf.plot(ax=ax)

In [ ]:
#@markdown ### **Uploading timeseries**
upload_timeseries = files.upload()
measurements = pd.read_csv(list(upload_timeseries.keys())[0])

tList = list(measurements.columns)
tList.append('NA')
ts_well_ID = widgets.Dropdown(options=tList, description = "Well ID")
ts_date = widgets.Dropdown(options=tList, description = "Date")
ts_measurement = widgets.Dropdown(options=tList, description = "Measurement")
ts_aquifer_ID = widgets.Dropdown(options=tList, description = "Aquifer ID")

tItems = [ts_well_ID, ts_date, ts_measurement, ts_aquifer_ID]
print("\nPlease select the appropriate headers for your file")
widgets.GridBox(tItems, layout=widgets.Layout(grid_template_columns="repeat(1, 550px)"))

In [ ]:
#To avoid type mismatch datatypes
if (ts_aquifer_ID.value !="NA"):
  measurements[ts_aquifer_ID.value] = measurements[ts_aquifer_ID.value].astype(str)

In [ ]:
#@markdown ### **Formatting Timeseries**
measurements[ts_date.value] = pd.to_datetime(measurements[ts_date.value], infer_datetime_format=True)
if (ts_aquifer_ID.value !="NA"):
  measurements = measurements[measurements[ts_aquifer_ID.value].isin(aquifer_ID)]
measurements

In [ ]:
#@markdown ### **Dropping measurements that don't have corresponding wells**
no_uniquewells = len(measurements[ts_well_ID.value].unique())

measurements = measurements[measurements[ts_well_ID.value].isin(wells_gdf[w_well_ID.value])].reset_index(drop=True)

no_uniquewellsref = len(measurements[ts_well_ID.value].unique())

print(no_uniquewells, ' unique well IDs')
print(no_uniquewellsref, ' wells with measurements')

In [ ]:
#@markdown ### **Defining aquifer bounds**
bbox = aquifer.bounds.values[0].tolist()
bbox

# Accessing Data from Server

In [ ]:
#@markdown ### **Defining functions for this section**

def get_pdsi_df(aquifer_gdf):
    data_dir = "/content"
    nc_file = os.path.join(data_dir, "pdsi.nc4")
    pdsi_ds = xarray.open_dataset(nc_file, decode_times=False)
    units, _, reference_date = pdsi_ds.time.attrs["units"].split("since")
    pdsi_ds["time"] = pd.date_range(
        start=reference_date, periods=pdsi_ds.sizes["time"], freq="MS"
    )
    ds = pdsi_ds["pdsi_filled"].to_dataset()
    ds["pdsi_filled"] = ds["pdsi_filled"].rio.write_crs("epsg:4326")
    ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    # aquifer_geom = wkt.loads(aquifer_obj[0])
    # aquifer_gdf = gpd.GeoDataFrame(
    #     {"name": ["random"], "geometry": [aquifer_geom]}, crs="EPSG:4326"
    # )
    clipped_ds = ds.rio.clip(
        aquifer_gdf.geometry.apply(mapping),
        aquifer_gdf.crs,
        drop=True,
        all_touched=True,
    ).drop("spatial_ref")
    filled_df = (
        clipped_ds.to_dataframe()
        .reset_index()
        .groupby("time")["pdsi_filled"]
        .mean()
        .reset_index()
        .set_index("time")
    )
    return filled_df

# outdated function sarva wanted replaced
def get_time_bounds(url):
    # This function returns the first and last available time
    # from a url of a getcapabilities page located on a Thredds Server
    f = urllib.request.urlopen(url)
    tree = ET.parse(f)
    root = tree.getroot()
    # These lines of code find the time dimension information for the netcdf on the Thredds server
    dim = root.findall(".//{http://www.opengis.net/wms}Dimension")
    dim = dim[0].text
    times = dim.split(",")
    times.pop(0)
    timemin = times[0]
    timemax = times[-1]
    # timemin and timemax are the first and last available times on the specified url
    return timemin, timemax
#
# # sarva's new function
# def get_time_bounds(url):
#     # This function returns the first and last available time
#     # from a url of a getcapabilities page located on a Thredds Server
#     f = urllib.request.urlopen(url)
#     tree = ET.parse(f)
#     root = tree.getroot()
#     # These lines of code find the time dimension information for the netcdf on the Thredds server
#     dim = root.findall(".//{http://www.opengis.net/wms}Dimension")
#     dim = dim[0].text
#     times = dim.split(",")
#     timemin = re.sub(r"[\n\t\s]*", "", times[0])
#     timemax = re.sub(r"[\n\t\s]*", "", times[-1])
#     # timemin and timemax are the first and last available times on the specified url
#     return timemin, timemax



def get_thredds_value(server, layer, bbox):
    # This function returns a pandas dataframe of the timeseries values of a specific layer
    # at a specific latitude and longitude from a file on a Thredds server
    # server: the url of the netcdf desired netcdf file on the Thredds server to read
    # layer: the name of the layer to extract timeseries information from for the netcdf file
    # lat: the latitude of the point at which to extract the timeseries
    # lon: the longitude of the point at which to extract the timeseries
    # returns df: a pandas dataframe of the timeseries at lat and lon for the layer in the server netcdf file
    # calls the getTimeBounds function to get the first and last available times for the netcdf file on the server
    time_min, time_max = get_time_bounds(
        server + "?service=WMS&version=1.3.0&request=GetCapabilities"
    )
    print(time_min)
    print(time_max)
    # These lines properly format a url request for the timeseries of a speific layer from a netcdf on
    # a Thredds server
    server = f"{server}?service=WMS&version=1.3.0&request=GetFeatureInfo&CRS=CRS:84&QUERY_LAYERS={layer}"
    server = f"{server}&X=0&Y=0&I=0&J=0&BBOX={bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}&LAYER={layer}"
    server = f"{server}&WIDTH=1&Height=1&INFO_FORMAT=text/xml&STYLES=raster/default"
    server = f"{server}&TIME={time_min}/{time_max}"
    print(server)
    f = request.urlopen(server)
    tree = ET.parse(f)
    root = tree.getroot()
    features = root.findall("FeatureInfo")
    times = []
    values = []
    for child in features:
        time = datetime.datetime.strptime(child[0].text, "%Y-%m-%dT%H:%M:%S.%fZ")
        times.append(time)
        values.append(child[1].text)

    df = pd.DataFrame(index=times, columns=[layer], data=values)
    df[layer] = df[layer].replace("none", np.nan).astype(float)
    return df

def sat_resample(gldas_df):
    # resamples the data from both datasets to a monthly value,
    # uses the mean of all measurements in a month
    # first resample to daily values, then take the start of the month
    gldas_df = gldas_df.resample("D").mean()
    # D is daily, mean averages any values in a given day, if no data in that day, gives NaN

    gldas_df.interpolate(method="pchip", inplace=True, limit_area="inside")

    gldas_df = gldas_df.resample("MS").first()
    # MS means "month start" or to the start of the month, this is the interpolated value

    return gldas_df

def sat_rolling_window(gldas_df):
    years = [1, 3, 5, 10]
    names = list(gldas_df.columns)
    new_names = copy.deepcopy(
        names
    )  # names is a list of the varibiles in the data frame, need to unlink for loop
    # This loop adds the yearly, 3-year, 5-year, and 10-year rolling averages of each variable to the dataframe
    # rolling averages uses the preceding time to compute the last value,
    # e.g., using the preceding 5 years of data to get todays
    for name in names:
        for year in years:
            new = name + "_yr" + str(year).zfill(2)
            gldas_df[new] = gldas_df[name].rolling(year * 12).mean()
            new_names.append(new)
    return gldas_df, new_names

In [ ]:
#@markdown ### **Downloading PDSI data from server**
pdsi_response = requests.get("https://github.com/BYU-Hydroinformatics/gwdm-data/blob/main/pdsi.nc4?raw=true")
with open("pdsi.nc4", "wb") as binary_file:
    # Write bytes to file
    binary_file.write(pdsi_response.content)
pdsi_df = get_pdsi_df(aquifer)

In [ ]:
#@markdown ### **Downloading soil moisture data from server**
SERVER2 = "https://tethyswa.servirglobal.net/thredds/wms/testAll/psl/soilw.mon.mean.nc"
#SERVER2 = "https://www.psl.noaa.gov/thredds/wms/Datasets/cpcsoil/soilw.mon.mean.nc"
LAYER2 = "soilw"  # name of data column to be returned
soilw_df = get_thredds_value(SERVER2, LAYER2, bbox)

In [ ]:
#@markdown ### **Resampling the data grid size and adding rolling average**
gldas_df = pd.concat([pdsi_df, soilw_df], join="outer", axis=1)
gldas_df = sat_resample(gldas_df)
gldas_df, names = sat_rolling_window(gldas_df)

# Temporal Interpolation

In [ ]:
#@markdown ### **Setting parameters for temporal interpolation**
min_samples = 5 #@param {type:"integer"}
gap_size = "3650" #@param {type:"string"}
pad = 365 #@param {type:"integer"}
spacing = "1MS"

In [ ]:
#@markdown ### **Defining functions for this section**

# def smooth(y, box_size):
#     """moving window function using convolution
#     y:          1D vector of values
#     box_size:   size of the moving window, should be an odd integer, will set if not
#     this function pads the data so that the edges are equal to the
#     orginal value
#     """
#     if int(box_size) % 2 == 0:  # make sure box_size is odd integer
#         box_size = int(box_size) + 1  # if even, add one
#     else:
#         box_size = int(box_size)  # if not even (e.g., float), make integer

#     box = np.ones(box_size) / box_size  # convolution kernal for moving window
#     y_pad = np.pad(
#         y, (box_size // 2, box_size - 1 - box_size // 2), mode="edge"
#     )  # padd for edge effects
#     y_smooth = np.convolve(
#         y_pad, box, mode="valid"
#     )  # convolve, 'valid' key work trims pad
#     return y_smooth

# def plot_anomalies(df, x='date', y='amount'):

#     # categories will be having values from 0 to n
#     # for each values in 0 to n it is mapped in colormap
#     categories = df['Predictions'].to_numpy()
#     colormap = np.array(['g', 'r'])

#     f = plt.figure(figsize=(12, 4))
#     f = plt.scatter(df[x], df[y], c=colormap[categories])
#     f = plt.xlabel(x)
#     f = plt.ylabel(y)
#     # f = plt.xticks(rotation=90)
#     plt.show()

# def extract_query_objects(region_id, aquifer_id, variable):
#     session = get_session_obj()
#     aquifer_obj = (
#         session.query(gf2.ST_AsText(Aquifer.geometry), Aquifer.aquifer_name)
#         .filter(Aquifer.region_id == region_id, Aquifer.id == aquifer_id)
#         .first()
#     )
#     bbox = wkt.loads(aquifer_obj[0]).bounds
#     wells_query = session.query(Well).filter(Well.aquifer_id == aquifer_id)
#     wells_query_df = pd.read_sql(wells_query.statement, session.bind)
#     well_ids = [int(well_id) for well_id in wells_query_df.id.values]
#     # well_dict = {well.id: well.gse for well in wells_query_df.itertuples()}
#     m_query = session.query(Measurement).filter(
#         Measurement.well_id.in_(well_ids), Measurement.variable_id == variable
#     )
#     measurements_df = pd.read_sql(m_query.statement, session.bind)
#     # measurements_df['gse'] = measurements_df['well_id'].map(well_dict)
#     measurements_df["date"] = pd.to_datetime(
#         measurements_df.ts_time, infer_datetime_format=True
#     )

#     session.close()

#     return bbox, wells_query_df, measurements_df, aquifer_obj    

def extract_well_data(name, well_df, min_samples=0, ts_col="ts_value", date_col="date"):
    if len(well_df) >= min_samples:
        # if (well_df['date'].min() < start_date) and (well_df['date'].max() > end_date):
        # elevation = well_df['gse'].unique()[0]
        df = pd.DataFrame(
            index=well_df[date_col].values,
            data=well_df[ts_col].values,
            columns=[name],
        )
        df = df[np.logical_not(df.index.duplicated())]
        return df

def interp_well(wells_df, gap_size, pad, spacing):
    well_interp_df = pd.DataFrame()
    # create a time index to interpolate over - cover entire range
    interp_index: pd.DatetimeIndex = pd.date_range(
        start=min(wells_df.index), freq=spacing, end=max(wells_df.index)
    )
    # loop over each well, interpolate data using pchip
    for well in wells_df:
        temp_df = wells_df[well].dropna()  # available data for a well

        x_index = temp_df.index.astype("int")  # dates for available data

        x_diff = temp_df.index.to_series().diff()  # data gap sizes

        fit2 = interpolate.pchip(x_index, temp_df)  # pchip fit to data

        ynew = fit2(interp_index.astype("int"))  # interpolated data on full range

        interp_df = pd.DataFrame(ynew, index=interp_index, columns=[well])

        # replace data in gaps of > 1 year with nans

        gaps = np.where(x_diff > gap_size)  # list of indexes where gaps are large

        for g in gaps[0]:
            start = x_diff.index[g - 1] + datetime.timedelta(days=pad)

            end = x_diff.index[g] - datetime.timedelta(days=pad)

            interp_df[start:end] = np.nan

        beg_meas_date = x_diff.index[0]  # date of 1st measured point

        end_meas_date = temp_df.index[-1]  # date of last measured point

        mask1 = (
            interp_df.index < beg_meas_date
        )  # locations of data before 1st measured point

        interp_df[mask1] = np.nan  # blank out data before 1st measured point

        mask2 = (
            interp_df.index >= end_meas_date
        )  # locations of data after the last measured point

        interp_df[mask2] = np.nan  # blank data from last measured point

        # add the interp_df data to the full data frame

        well_interp_df = pd.concat(
            [well_interp_df, interp_df], join="outer", axis=1, sort=False
        )
    return well_interp_df

def norm_training_data(in_df, ref_df):
    norm_in_df = (in_df - ref_df.min().values) / (
        ref_df.max().values - ref_df.min().values
    )  # use values as df sometimes goofs
    return norm_in_df

def input_to_hidden(x, Win, b):
    # setup matrixes
    a = np.dot(x, Win) + b
    a = np.maximum(a, 0, a)  # relu
    return a

def predict(in_values, W_in, b, W_out):
    x = input_to_hidden(in_values, W_in, b)
    y = np.dot(x, W_out)
    return y

def impute_data(comb_df, well_names, names):
    # for out test set we will impute everything
    imputed_df = pd.DataFrame(index=comb_df.index)

    for well in well_names:  # list of the wells in the aquifer
        train_nona_df = comb_df.dropna(
            subset=[well]
        )  # drop any rows with na in well (measured) data
        labels_df = train_nona_df[
            well
        ]  # measured data used as "labels" or truth in training
        tx_df = train_nona_df[
            names
        ]  # data we will predict with only over the test period
        all_tx_df = comb_df[names]  # data over the full period, will use for imputation

        tx = tx_df.values  # convert to an array
        x1 = np.column_stack(np.ones(tx.shape[0])).T  # bias vector of 1's
        tx = np.hstack((tx, x1))  # training matrix
        ty = labels_df.values
        input_length = tx.shape[1]
        hidden_units = 500
        lamb_value = 100
        W_in = np.random.normal(size=[input_length, hidden_units])
        b = np.random.normal(size=[hidden_units])

        # now do the matrix multiplication
        X = input_to_hidden(
            tx, W_in, b
        )  # setup matrix for multiplication, it is a function
        I = np.identity(X.shape[1])
        I[X.shape[1] - 1, X.shape[1] - 1] = 0
        I[X.shape[1] - 2, X.shape[1] - 2] = 0
        W_out = np.linalg.lstsq(X.T.dot(X) + lamb_value * I, X.T.dot(ty), rcond=-1)[0]
        all_tx_values = all_tx_df.values
        a1 = np.column_stack(np.ones(all_tx_values.shape[0])).T
        all_tx_values = np.hstack((all_tx_values, a1))
        predict_values = predict(all_tx_values, W_in, b, W_out)  # it is a function
        #pre_name = f"{well}_imputed"
        pre_name = f"{well}"
        imputed_df[pre_name] = pd.Series(predict_values, index=imputed_df.index)

    return imputed_df

def renorm_data(in_df, ref_df):
    assert in_df.shape[1] == ref_df.shape[1], "must have same # of columns"
    renorm_df = (
        in_df * (ref_df.max().values - ref_df.min().values)
    ) + ref_df.min().values
    return renorm_df

In [ ]:
#@markdown ### **Collect well data that qualifies for use in interpolation**
wells_df = pd.concat([
        extract_well_data(name, group, min_samples, ts_measurement.value, ts_date.value)
        for name, group in measurements.groupby(ts_well_ID.value)],
    axis=1, sort=False,)

In [ ]:
#@markdown ### **Cleaning up wells dataframe**
wells_df.drop_duplicates(inplace=True)
wells_df[wells_df == 0] = np.nan
# wells_df.to_csv("wells_one.csv")
wells_df.dropna(thresh=min_samples, axis=1, inplace=True)
no_qualifiedwells = len(wells_df.columns.values)
no_droppedwells = no_uniquewells - no_qualifiedwells
print(no_uniquewellsref,  ' wells with measurements')
print(no_droppedwells,  ' wells dropped')
print(no_qualifiedwells,  ' qualified for use in interpolation')

In [ ]:
#@markdown ### **Running simple pchip temporal interpolation**
well_interp_df = interp_well(wells_df, gap_size, pad, spacing)
well_interp_df.dropna(thresh=min_samples, axis=1, inplace=True)

In [ ]:
#@markdown ### **Running machine learning algorithm to fill gaps**
# combine the  data from the wells and the satellite observations  to a single dataframe (combined_df)
# this will have a row for every measurement (on the start of the month) a column for each well,
# and a column for pdsi and soilw and their rolling averages, and potentially offsets
combined_df = pd.concat(
    [well_interp_df, gldas_df], join="outer", axis=1, sort=False
)
combined_df.dropna(
    subset=names, inplace=True
)  # drop rows where there are no satellite data
combined_df.dropna(how="all", axis=1, inplace=True)

norm_df = norm_training_data(combined_df, combined_df)
norm_df.dropna(how="all", axis=1, inplace=True)
well_names = [col for col in well_interp_df.columns if col in norm_df.columns]
imputed_norm_df = impute_data(norm_df, well_names, names)
ref_df = combined_df[well_names]
imputed_df = renorm_data(imputed_norm_df, ref_df)

imputed_well_names = imputed_df.columns  # create a list of well names
loc_well_names = [(strg.replace("_imputed", "")) for strg in imputed_well_names]  # strip off "_imputed"

In [ ]:
#@markdown ### **Merging measurement, pchip, and machine learning dataframes**
#formating column names to match so that the dataframes can be combined
well_interp_columns = []
for i in range(len(well_interp_df.columns)) : well_interp_columns.append(str(well_interp_df.columns[i]))
well_interp_df.columns = well_interp_columns

imputed_columns = []
for i in range(len(imputed_df.columns)) : imputed_columns.append(str(imputed_df.columns[i]))
imputed_df.columns = imputed_columns

#replacing the machine learning values with pchip values when available
replaced_df = well_interp_df.combine_first(imputed_df)


In [ ]:
#@markdown ### **Run Cell and Select the Well you Wish to Plot**
pList = imputed_df.columns
p_index = widgets.Dropdown(options=pList, description = "Well Index")

pItems = [p_index]
print("\nSelect the well index you wish to plot")
widgets.GridBox(pItems, layout=widgets.Layout(grid_template_columns="repeat(1, 550px)"))

In [ ]:
#@markdown ### **Plots measurments, pchip, ml timeseries to show complete temporal interpolation**
well_no = imputed_df.columns.get_loc(p_index.value)

#formatting dates for plotting
imputed_date =  pd.to_datetime(imputed_df.index.values)
interp_date =  pd.to_datetime(well_interp_df.index.values)
wells_date =  pd.to_datetime(wells_df.index.values)
replaced_date =  pd.to_datetime(replaced_df.index.values)

#ignoring NANs for plotting
well_interp_df.dropna()
wells_df.dropna()

#plotting the three dataframes: measurements, pchip, and machine learning
fig, ax = plt.subplots(2, 1, figsize=(25,10))
#ax.set_xlim([datetime.date(2001, 1, 26), datetime.date(2021, 2, 1)])
ax[0].plot(imputed_date, imputed_df[imputed_df.columns[well_no]]) #machine learning
ax[0].plot(interp_date, well_interp_df[well_interp_df.columns[well_no]], color = 'red') #pchip
ax[0].plot(wells_date, wells_df[wells_df.columns[well_no]], marker = "o", linestyle = 'none', color = 'green') #measurements
ax[1].plot(replaced_date, replaced_df[replaced_df.columns[well_no]], color = 'purple')

# Spatial Interpolation

In [ ]:
#@markdown ### **Setting Parameters for Spatial Interpolation**
raster_extent = "aquifer" #@param ["aquifer", "wells"]
raster_interval = 12 #@param {type:"slider", min:1, max:24, step:1}
output_file = "results.nc" #@param {type:"string"}
aquifer_name = "sunflower" #@param {type:"string"}
units = "acre-ft" #@param ["acre-ft", "cubic-ft", "Cubic Meter"]
start_date = 2001
end_date = 2019
#let user pick the entire date
#then if they pick an inappropriate date, give them a suggestion for the date

In [ ]:
#@markdown ### **Defining functions for this section**

def create_grid_coords(x_c, y_c, x_steps, bbox, raster_extent):
    # create grid coordinates fro kriging, make x and y steps the same
    # x_steps is the number of cells in the x-direction
    min_x = min_y = max_x = max_y = None
    if raster_extent == "aquifer":
        min_x, min_y, max_x, max_y = bbox
    elif raster_extent == "wells":
        min_x, max_x = min(x_c), max(x_c)
        min_y, max_y = min(y_c), max(y_c)

    n_bin = np.absolute((max_x - min_x) / x_steps)  # determine step size (positive)
    # make grid 10 bin steps bigger than date, will give 110 steps in x-direction
    grid_x = np.arange(
        min_x - 5 * n_bin, max_x + 5 * n_bin, n_bin
    )  # make grid 10 steps bigger than data
    grid_y = np.arange(
        min_y - 5 * n_bin, max_y + 5 * n_bin, n_bin
    )  # make grid 10 steps bigger than data

    return grid_x, grid_y

def krig_field_generate(var_fitted, x_c, y_c, values, grid_x, grid_y):
    # use GSTools to krig  the well data, need coords and value for each well
    # use model variogram paramters generated by GSTools
    # fast - is faster the variogram fitting
    krig_map = gs.krige.Ordinary(var_fitted, cond_pos=[x_c, y_c], cond_val=values)
    krig_map.structured([grid_x, grid_y], chunk_size=1000)  # krig_map.field is the numpy array of values
    return krig_map    

def fit_model_var(x_c, y_c, values, bbox, raster_extent):
    # fit the model varigrom to the experimental variogram
    min_x = min_y = max_x = max_y = None
    if raster_extent == "aquifer":
        min_x, min_y, max_x, max_y = bbox
    elif raster_extent == "wells":
        min_x, max_x = min(x_c), max(x_c)
        min_y, max_y = min(y_c), max(y_c)

    # first get the coords and determine distances
    x_delta = max_x - min_x  # distance across x coords
    y_delta = max_y - min_y  # distance across y coords
    max_dist = (
        np.sqrt(np.square(x_delta + y_delta)) / 4
    )  # assume correlated over 1/4 of distance
    data_var = np.var(values)
    data_std = np.std(values)
    fit_var = gs.Stable(dim=2, var=data_var, len_scale=max_dist, nugget=data_std)
    return fit_var

def generate_nc_file(
    file_name, grid_x, grid_y, years_df, x_coords, y_coords, bbox, raster_extent
):
    temp_dir = tempfile.mkdtemp()
    file_path = os.path.join(temp_dir, file_name)
    h = netCDF4.Dataset(file_path, "w", format="NETCDF4")
    lat_len = len(grid_y)
    lon_len = len(grid_x)
    time_dim = h.createDimension("time", 0)
    lat = h.createDimension("lat", lat_len)
    lon = h.createDimension("lon", lon_len)
    latitude = h.createVariable("lat", np.float64, ("lat"))
    longitude = h.createVariable("lon", np.float64, ("lon"))
    time_dim = h.createVariable("time", np.float64, ("time"), fill_value="NaN")
    ts_value = h.createVariable(
        "tsvalue", np.float64, ("time", "lon", "lat"), fill_value=-9999
    )
    latitude.long_name = "Latitude"
    latitude.units = "degrees_north"
    latitude.axis = "Y"
    longitude.long_name = "Longitude"
    longitude.units = "degrees_east"
    longitude.axis = "X"
    time_dim.axis = "T"
    time_dim.units = "days since 0001-01-01 00:00:00 UTC"

    latitude[:] = grid_y[:]
    longitude[:] = grid_x[:]

    time_counter = 0
    for measurement in years_df:
        # loop through the data
        values = years_df[measurement].values

        beg_time = timer()  # time the kriging method including variogram fitting
        # fit the model variogram to the experimental variogram
        var_fitted = fit_model_var(
            x_coords, y_coords, values, bbox, raster_extent
        )  # fit variogram
        krig_map = krig_field_generate(
            var_fitted, x_coords, y_coords, values, grid_x, grid_y
        )  # krig data
        # krig_map.field provides the 2D array of values
        end_time = timer()
        time_dim[time_counter] = measurement.toordinal()
        ts_value[time_counter, :, :] = krig_map.field
        time_counter += 1

    h.close()
    return Path(file_path)

def earth_radius(lat):
    """
    calculate radius of Earth assuming oblate spheroid
    defined by WGS84
    Input
    ---------
    lat: vector or latitudes in degrees
    Output
    ----------
    r: vector of radius in meters
    Notes
    -----------
    WGS84: https://earth-info.nga.mil/GandG/publications/tr8350.2/tr8350.2-a/Chapter%203.pdf
    Taken from: https://gist.github.com/lgloege/3fdb1ed83b002d68d8944539a797b0bc
    """
    from numpy import deg2rad

    # define oblate spheroid from WGS84
    a = 6378137
    b = 6356752.3142
    e2 = 1 - (b ** 2 / a ** 2)

    # convert from geodecic to geocentric
    # see equation 3-110 in WGS84
    lat = deg2rad(lat)
    lat_gc = np.arctan((1 - e2) * np.tan(lat))

    # radius equation
    # see equation 3-107 in WGS84
    r = (a * (1 - e2) ** 0.5) / (1 - (e2 * np.cos(lat_gc) ** 2)) ** 0.5

    return r

def calculate_aquifer_area(imputed_raster, units):
    y_res = abs(
        round(imputed_raster["lat"].values[0] - imputed_raster["lat"].values[1], 7)
    )  # this assumes all cells will be the same
    # size in one dimension (all cells will have same x-component)
    x_res = abs(
        round(imputed_raster["lon"].values[0] - imputed_raster["lon"].values[1], 7)
    )
    area = 0
    # Loop through each y row
    for y in range(imputed_raster.lat.size):
        # Define the upper and lower bounds of the row
        cur_lat_max = math.radians(imputed_raster["lat"].values[y] + (y_res / 2))
        cur_lat_min = math.radians(imputed_raster["lat"].values[y] - (y_res / 2))

        # Count how many cells in each row are in aquifer (i.e. and, therefore, not nan)
        x_count = np.count_nonzero(~np.isnan(imputed_raster.tsvalue[0, :, y]))

        # Area calculated based on the equation found here:
        # https://www.pmel.noaa.gov/maillists/tmap/ferret_users/fu_2004/msg00023.html
        #     (pi/180) * R^2 * |lon1-lon2| * |sin(lat1)-sin(lat2)|
        radius = earth_radius(imputed_raster["lat"].values[y])
        if units == "acre-ft":
            area_factor = 1 / 4046.8564224
        elif units == "cubic-ft":
            area_factor = 10.764
        else:
            area_factor = 1
        area += (
            radius ** 2
            * area_factor
            * math.radians(x_res * x_count)
            * abs((math.sin(cur_lat_min) - math.sin(cur_lat_max)))
        )

    return area

In [ ]:
#@markdown ### **Setting up grid for nc file**
coords_df = wells_gdf[wells_gdf[w_well_ID.value].astype(str).isin(loc_well_names)]
x_coords = coords_df[w_long.value].values
y_coords = coords_df[w_lat.value].values

x_steps = 150  # steps in x-direction, number of y-steps will be computed with same spacing, adds 10%
grid_x, grid_y = create_grid_coords(
    x_coords, y_coords, x_steps, bbox, raster_extent
)  # coordinates for x and y axis - not full grid
replaced_df = replaced_df[
    (replaced_df.index >= f"04-01-{start_date-1}")
    & (replaced_df.index <= f"12-31-{end_date+1}")
]
# skip_month = 48  # take data every nth month (skip_months), e.g., 60 = every 5 years
years_df = replaced_df.iloc[
    ::raster_interval
].T  # extract every nth month of data and transpose array

file_name = f"{aquifer_name}_{time.time()}.nc"
# setup a netcdf file to store the time series of rasters

In [ ]:
#@markdown ### **Generate nc file using kriging**
nc_file_path = generate_nc_file(file_name, grid_x, grid_y, years_df, x_coords, y_coords, bbox, raster_extent)

In [ ]:
#@markdown ### **Assigning dimensions and clipping nc file**
file_path = nc_file_path
temp_dir = file_path.parent.absolute()
interp_nc = xarray.open_dataset(file_path)
interp_nc.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
interp_nc.rio.write_crs("epsg:4326", inplace=True)
clipped_nc = interp_nc.rio.clip(aquifer.geometry.apply(mapping), crs=4326, drop=True)

In [ ]:
#@markdown ### **Calculating drawdown**
area = calculate_aquifer_area(clipped_nc, units)
if units == "acre-ft":
    vol_unit = "Acre Feet"
elif units == "cubic-ft":
    vol_unit = "Cubic Feet"
else:
    vol_unit = "Cubic Meters"
# Calculate total drawdown volume at each time step
drawdown_grid = np.zeros(
    (clipped_nc.time.size, clipped_nc.lon.size, clipped_nc.lat.size)
)
drawdown_volume = np.zeros(clipped_nc.time.size)
for t in range(clipped_nc.time.size):
    # Calculate drawdown at time t by subtracting original WTE at time 0
    drawdown_grid[t, :, :] = (
        clipped_nc["tsvalue"][t, :, :] - clipped_nc["tsvalue"][0, :, :]
    )
    # Average drawdown across entire aquifer x storage_coefficient x area of aquifer
    drawdown_volume[t] = np.nanmean(
        drawdown_grid[t, :, :] * 0.1 * area
    )
print(area, units)
print(type(clipped_nc))
clipped_nc.attrs = {"area": area, "area_units": units}
clipped_nc["drawdown"] = (["time", "lon", "lat"], drawdown_grid)
clipped_nc["volume"] = (["time"], drawdown_volume, {"units": vol_unit})

In [ ]:
#@markdown ### **Saving to notebook**
clipped_nc.to_netcdf(aquifer_name + "_"+ output_file)

# Visualizing Results

In [ ]:
#@markdown ### **Plotting the Drawdown Curve**
nc = xarray.open_dataset('/content/'+aquifer_name + "_"+ output_file)
time = nc.variables['time'][:]
volume = nc.variables['volume'][:]
ts = nc.variables['tsvalue'][:]

fig, ax = plt.subplots(1, 1, figsize=(25,5))
ax.plot(time, volume)
plt.xlabel("Date (years)")
plt.ylabel("Volume ("+ vol_unit+")")
plt.title(aquifer_name.capitalize() + " Drawdown Curve")
### don't clip it so we can see the whole curve from machine learning

In [ ]:
#Export drawdown curve to CSV
drawdown_df = pd.DataFrame({'Time': time, 'Volume': volume})
drawdown_df.to_csv(aquifer_name + "_" + "drawdown.csv")

In [ ]:
%%capture
!pip install basemap
from mpl_toolkits.basemap import Basemap
import numpy
from os import path
import glob
from PIL import Image

In [ ]:
leftbound = bbox[0]
bottombound = bbox[1]
rightbound = bbox[2]
topbound = bbox[3]
Frame_Cushion = 0.25

In [ ]:
#Set up the time index for your GIF
starttime = numpy.datetime64(f"{start_date+1}-04-01", 'M')
endtime = numpy.datetime64(f"{end_date-1}-12-31", 'M')
startindex, endindex = -1, -1
for i in range(len(nc['time'][:])-1):
  check = numpy.datetime64(nc.coords['time'].values[i], 'M')
  if starttime < check and startindex == -1:
    startindex = i-1
  if endtime < check and endindex == -1: 
    endindex = i+1

In [ ]:
%%capture
if path.exists('/content/images_gif') == False:
  os.mkdir('/content/images_gif')

fig, ax = plt.subplots()
aquifer.plot(color="none", edgecolor="red", ax=ax)
m = Basemap(projection='cyl', llcrnrlon=leftbound-Frame_Cushion, llcrnrlat=bottombound-Frame_Cushion, urcrnrlon=rightbound+Frame_Cushion, urcrnrlat=topbound+Frame_Cushion, resolution='i', epsg=4326)
EsriBasemaps = "World_Topo_Map" #@param ["NatGeo_World_Map", "USA_Topo_Maps", "World_Imagery", "World_Physical_Map", "World_Shaded_Relief", "World_Street_Map", "World_Terrain_Base", "World_Topo_Map"]
m.arcgisimage(server='http://server.arcgisonline.com/ArcGIS', service=EsriBasemaps, xpixels = 400, dpi = 69,verbose = True)

for i in range(startindex,endindex):
  im = plt.imshow(numpy.rot90(ts[i,:,:]), cmap="Spectral",extent=(leftbound, rightbound, bottombound, topbound) ,vmin=numpy.amin(ts),vmax=numpy.amax(ts))
  plt.title(aquifer_name.capitalize()+" "+str(nc.coords['time'].values[i])[:10], pad = 15)
  if i==startindex:  
    cbar = plt.colorbar()
    cbar.set_label(ts_measurement.value+" in "+units, labelpad = 20)
  plt.savefig(f"/content/images_gif/image'{str(nc.coords['time'].values[i])[:10]}.jpg")

In [ ]:
frame_duration=300
frames = []
imgs=glob.glob(f"/content/images_gif/*.jpg")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
    frames[0].save(aquifer_name + '_animation.gif', format='GIF', append_images=frames[1:],save_all=True, duration = frame_duration, loop=0)